# 🔧 Debug Drill: The Backward Coefficient

**Scenario:**
You trained a logistic regression model for churn prediction. Your PM looks at the coefficients and is confused.

"Wait," they say. "The model says longer tenure INCREASES churn? That doesn't make sense!"

**Your Task:**
1. Run the notebook and see the strange coefficient
2. Diagnose why the coefficient has the wrong sign
3. Fix it
4. Write a 3-bullet postmortem

---

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# Load data
try:
    df = pd.read_csv('https://raw.githubusercontent.com/189investmentai/ml-foundations-interactive/main/streamcart_customers.csv')
except:
    df = pd.read_csv('../../data/streamcart_customers.csv')

print(f"Loaded {len(df):,} customers")

In [ ]:
# ===== BUGGY CODE =====

# Calculate months until next anniversary
df['months_until_anniversary'] = 12 - (df['tenure_months'] % 12)

# Features including the new one
features = [
    'tenure_months',
    'months_until_anniversary',  # <-- BUG: This is derived from tenure_months!
    'logins_last_30d',
    'support_tickets_last_30d',
]

X = df[features].fillna(0)
y = df['churn_30d']

# Split and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Look at coefficients
print("=== Coefficients ===")
for feature, coef in zip(features, model.coef_[0]):
    direction = "↑ churn" if coef > 0 else "↓ churn"
    print(f"{feature:30} {coef:+.4f}  ({direction})")

In [ ]:
# The PM's concern:
print("\n❓ PM's Question:")
print("'Why does tenure_months have a POSITIVE coefficient?'")
print("'Longer customers should be LESS likely to churn, not more!'")

---

## Your Investigation

### Step 1: Understand the problem

In [ ]:
# TODO: Check the correlation between the features

print("=== Feature Correlations ===")
print(df[features].corr())

In [ ]:
# Look at the relationship between tenure and anniversary
print("\n=== How are tenure and anniversary related? ===")
print(df[['tenure_months', 'months_until_anniversary']].head(10))

# What's the correlation?
corr = df['tenure_months'].corr(df['months_until_anniversary'])
print(f"\nCorrelation: {corr:.3f}")

In [ ]:
# What do you notice? Write your diagnosis:

diagnosis = """
YOUR DIAGNOSIS:

The problem is called: _____________ (hint: multi-something)

Why does it cause weird coefficients?

"""
print(diagnosis)

### Step 2: Fix the code

In [ ]:
# TODO: Remove the redundant feature

features_fixed = [
    # List features without the multicollinearity
]

# Retrain
# X_fixed = ...
# model_fixed = ...

# Check new coefficients
# print("=== Fixed Coefficients ===")
# for feature, coef in zip(features_fixed, model_fixed.coef_[0]):
#     direction = "↑ churn" if coef > 0 else "↓ churn"
#     print(f"{feature:30} {coef:+.4f}  ({direction})")

In [ ]:
# ============================================
# SELF-CHECK
# ============================================

# After fixing, tenure should have a NEGATIVE coefficient
# (longer tenure = less churn)

# tenure_coef = model_fixed.coef_[0][features_fixed.index('tenure_months')]
# assert tenure_coef < 0, f"Tenure coefficient should be negative, got {tenure_coef:.4f}"
# print(f"✓ Tenure coefficient is now {tenure_coef:.4f} (negative = less churn)")

### Step 3: Write your postmortem

In [ ]:
postmortem = """
## Postmortem: Backward Coefficient Bug

### What happened:
- (Your answer)

### Root cause:
- (Your answer - mention multicollinearity)

### How to prevent:
- (Your answer)
"""

print(postmortem)

---

## ✅ Drill Complete!

**Key lesson:** When features are highly correlated (multicollinearity), coefficients become unstable and can even flip signs. Always check correlations before interpreting coefficients.